In [8]:
import pandas as pd
from OrcFxAPI import*
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass

In [ ]:
#get umbilical information
umb_OD = 0.215 #m
umb_EA = 773   #MN
E_Mod = 200e3 #MPA

In [ ]:
#get tube data
SMYS = 620 #MPa
tube_ID = 0.0381 #m
wall_thickness = 0.004 #m

p_i = 68.9 #MPA
p_o = 0.0  #MPA

tube_OD = tube_ID + 2.0*wall_thickness

In [ ]:
#project specific data
wd = 1790 #m
condition = "Normal" #possible values: "Normal" or "Abnormal"

if condition == "Normal":
    tau = 0.8
elif condition == "Abnormal":
    tau = 1.0

In [43]:
@dataclass
class Field:
    """ class to store field properties like depth """
    name:float = "Yellowtail"
    wd:float = 1790
    fpso:str = "One Guayana"
    numb_umbilicals:float = 3.0


@dataclass
class umb:
    """ class to define umbilical properties """
    name:str = "Umb 1 Yellowtail, static part"
    od:float = 0.215
    ea:float = 773.0
    e_mod:float = 200e3


@dataclass
class tube:
    """ class to define the tube properties"""
    name:str = '2.5in tube'
    id:float = 0.0381
    od:float = 0.0381 + 2*0.004
    wt:float = 0.004
    pi:float = 68.9
    po:float = 0.0


In [70]:
### defining the stress class
class stress():
    """ class to calculate stresses """

    def __init__(self, umb, tube):
        """ load the tube and umbilical classes to the stress class upon creation """
        self.tube = tube
        self.umb = umb
        
        # self.mbr = mbr
        # self.tens = tens

    #end cap stress
    def stress_aec(self) -> float:
        """ calculates the end cap stress, not dependent on the tension vs MBR values """
        return ((self.tube.pi-self.tube.po)*(self.tube.od-2.0*self.tube.wt)**2) / (self.tube.od**2 - (self.tube.od - 2*self.tube.wt)**2)

    def stress_at(self, tens) -> float:
        """ calculates the stress due to tensile loads """
        return (self.umb.e_mod * 0.001 * tens/self.umb.ea)

tube1 = tube()
umb1 = umb()
tube1.pi = 150
stress1 = stress(umb1,tube1)

stress1.stress_aec()
stress1.stress_at(150)

38.809831824062094

In [5]:
#STRESS FUNCTIONS


#end cap stress
def stress_aec() -> float:
    """ calculates the end cap stress, not dependent on the tension vs MBR values """
    return ((p_i-p_o)*(tube_OD-2.0*wall_thickness)**2) / (tube_OD**2 - (tube_OD - 2*wall_thickness)**2)


#stress due to tensile loads
def stress_at(tens) -> float:
    """ calculates the stress due to tensile loads """
    return (E_Mod * 0.001 * tens/umb_EA)


# INSIDE WALL
# stress inside the wall
def stress_bi(mbr) -> float:
    """ Calculates the stress due to bending inside the wall of the tube """
    return (E_Mod * (0.5 * tube_ID) / mbr)


#hoop stress inside the wall
def stress_hi() -> float:
    """ calculates teh hoop stress inside the wall """
    return (((p_i - p_o)*(tube_OD**2.0 + tube_ID**2.0)) / (tube_OD**2.0 - tube_ID**2.0)-p_o)


#radial strees inside the wall
def stress_ri() -> float:
    """ Calculates the radial stress inside the wall """
    return -p_i


#total axial stress inside
def total_axial_stress_inside(mbr, tens) -> float:
    """ Caculates the total axial stress inside the wall """
    return stress_aec() + stress_at(tens) + stress_bi(mbr)

#equivalent stress inside
def stress_eqi(mbr, tens) -> float:
    """ Calculates the inside equivalent stress """
    return np.sqrt((stress_hi()-total_axial_stress_inside(mbr,tens))**2.0 + (total_axial_stress_inside(mbr,tens)-stress_ri())**2.0 + (stress_ri() - stress_hi())**2.0) / np.sqrt(2.0)



#OUTSIDE WALL
#stress inside the wall
def stress_bo(mbr) -> float:
    """ Calculates the stress due to bending inside the wall of the tube """
    return E_Mod * (0.5 * tube_OD) / mbr


#hoop stress inside the wall
def stress_ho() -> float:
    """ calculates teh hoop stress inside the wall """
    return (p_i - p_o)*((2.0*tube_ID**2.0) / (tube_OD**2.0 - tube_ID**2.0)) -p_o


#radial strees inside the wall
def stress_ro() -> float:
    """ Calculates the radial stress inside the wall """
    return -p_o

#total axial stress inside
def total_axial_stress_outside(mbr, tens) -> float:
    """ Caculates the total axial stress inside the wall """
    return stress_aec() + stress_at(tens) + stress_bo(mbr)

#equivalent stress inside
def stress_eqo(mbr, tens) -> float:
    """ Calculates the inside equivalent stress """
    return np.sqrt((stress_ho()-total_axial_stress_outside(mbr,tens))**2.0 + (total_axial_stress_outside(mbr,tens)-stress_ro())**2.0 + (stress_ro() - stress_ho())**2.0) / np.sqrt(2.0)

#calculate the utilizations
def util(mbr,tens, type = 'normal') -> float:
    stress = max(stress_eqo(mbr,tens),stress_eqi(mbr,tens))
    if type == 'normal':
        return stress / SMYS
    else:
        return stress / (0.8 * SMYS)


In [6]:
if __name__ == '__main__':
    i_file = open('input_data.csv')
    data = i_file.readlines()
    i_file.close()

    mbr=[]
    tens=[]
    for line in data[1:]:
        mbr.append(float(line.split(',')[0]))
        tens.append(float(line.split(',')[1]))

    max_stress = [max(stress_eqi(mbr[i],tens[i]),stress_eqo(mbr[i],tens[i])) for i in range(len(mbr))]
    print(max_stress)

[495.9981631907829, 495.9669924904027, 496.0238539097665, 495.9409164999949, 495.96901701811953, 495.9238071295574, 495.8878392220522, 496.4287744506817, 496.21553694695797, 495.9329088127436, 495.60337847512443, 495.72602023357416, 496.2402860361266]
